In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%set_env CUDA_LAUNCH_BLOCKING=1

env: CUDA_LAUNCH_BLOCKING=1


In [3]:
import sys
import collections
import cv2
import os
import math
import random
import pickle
import copy
import numpy as np
sys.path.append("/home/tanvikamble/MeronymNet-PyTorch/src")

In [4]:
import torch
import torch.optim as optim
from torch_geometric.data import Data
from torch_geometric.utils import dense_to_sparse
from torch_geometric.data import DataLoader
import torch.utils.data as data_utils
from torch.utils.tensorboard import SummaryWriter
from losses import BoxVAE_losses as loss


from components.AutoEncoder import GCNAutoEncoder_Combined_Parts
from components.AutoEncoder import GCNAutoEncoder
from components.Decoder import Decoder
import matplotlib.pyplot as plt
# from mask_generation import masked_sketch

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [6]:
colors = [(1, 0, 0),
          (0.737, 0.561, 0.561),
          (0.255, 0.412, 0.882),
          (0.545, 0.271, 0.0745),
          (0.98, 0.502, 0.447),
          (0.98, 0.643, 0.376),
          (0.18, 0.545, 0.341),
          (0.502, 0, 0.502),
          (0.627, 0.322, 0.176),
          (0.753, 0.753, 0.753),
          (0.529, 0.808, 0.922),
          (0.416, 0.353, 0.804),
          (0.439, 0.502, 0.565),
          (0.784, 0.302, 0.565),
          (0.867, 0.627, 0.867),
          (0, 1, 0.498),
          (0.275, 0.51, 0.706),
          (0.824, 0.706, 0.549),
          (0, 0.502, 0.502),
          (0.847, 0.749, 0.847),
          (1, 0.388, 0.278),
          (0.251, 0.878, 0.816),
          (0.933, 0.51, 0.933),
          (0.961, 0.871, 0.702)]
colors = (np.asarray(colors)*255)
canvas_size = 660

def plot_bbx(bbx):
    bbx = bbx*canvas_size
    canvas = np.ones((canvas_size,canvas_size,3), np.uint8) * 255
    for i, coord in enumerate(bbx):
        x_minp, y_minp,x_maxp , y_maxp= coord
        if [x_minp, y_minp,x_maxp , y_maxp]!=[0,0,0,0]:
            cv2.rectangle(canvas, (int(x_minp), int(y_minp)), (int(x_maxp) , int(y_maxp) ), colors[i], 6)
    return canvas

# def plot_bbx(bbx):
#     bbx = bbx*canvas_size
#     canvas = np.ones((canvas_size,canvas_size,3), np.uint8) * 255
#     for i, coord in enumerate(bbx):
#         x, y, w ,h = coord
#         if [x, y, w ,h]!=[0,0,0,0]:
#             cv2.rectangle(canvas, (int(x), int(y)), (int(x + w) , int(y + h) ), colors[i], 6)
#     return canvas


def inference(decoder, nodes, obj_class, latent_dims, batch_size):

    decoder.cuda()
    z_latent = torch.normal(torch.zeros([batch_size,latent_dims])).cuda()
    nodes = torch.reshape(nodes,(batch_size,decoder.num_nodes))
    obj_class = torch.reshape(obj_class,(batch_size, decoder.class_size))
    conditioned_z = torch.cat([nodes, z_latent],dim=-1)
    conditioned_z = torch.cat([obj_class, conditioned_z],dim=-1)
    
    x_bbx, x_lbl = decoder(conditioned_z)
    if x_lbl == None:
        return x_bbx,z_latent
    else:
        return x_bbx, x_lbl, z_latent

In [7]:
bird_labels = {'head':1, 'leye':2, 'reye':3, 'beak':4, 'torso':5, 'neck':6, 'lwing':7, 'rwing':8, 'lleg':9, 'lfoot':10, 'rleg':11, 'rfoot':12, 'tail':13}


cat_labels = {'head':1, 'leye':2, 'reye':3, 'lear':4, 'rear':5, 'nose':6, 'torso':7, 'neck':8, 'lfleg':9, 'lfpa':10, 'rfleg':11, 'rfpa':12, 'lbleg':13, 'lbpa':14, 'rbleg':15, 'rbpa':16, 'tail':17}


cow_labels = {'head':1, 'leye':2, 'reye':3, 'lear':4, 'rear':5, 'muzzle':6, 'lhorn':7, 'rhorn':8, 'torso':9, 'neck':10, 'lfuleg':11, 'lflleg':12, 'rfuleg':13, 'rflleg':14, 'lbuleg':15, 'lblleg':16, 'rbuleg':17, 'rblleg':18, 'tail':19}



dog_labels = {'head':1, 'leye':2, 'reye':3, 'lear':4, 'rear':5, 'nose':6, 'torso':7, 'neck':8, 'lfleg':9, 'lfpa':10, 'rfleg':11, 'rfpa':12, 'lbleg':13, 'lbpa':14, 'rbleg':15, 'rbpa':16, 'tail':17, 'muzzle':18}


horse_labels = {'head':1, 'leye':2, 'reye':3, 'lear':4, 'rear':5, 'muzzle':6, 'lfho':7, 'rfho':8, 'torso':9, 'neck':10, 'lfuleg':11, 'lflleg':12, 'rfuleg':13, 'rflleg':14, 'lbuleg':15, 'lblleg':16, 'rbuleg':17, 'rblleg':18, 'tail':19, 'lbho':20, 'rbho':21}


person_labels = {'head':1, 'leye':2, 'reye':3, 'lear':4, 'rear':5, 'lebrow':6, 'rebrow':7, 'nose':8, 'mouth':9, 'hair':10, 'torso':11, 'neck': 12, 'llarm': 13, 'luarm': 14, 'lhand': 15, 'rlarm':16, 'ruarm':17, 'rhand': 18, 'llleg': 19, 'luleg':20, 'lfoot':21, 'rlleg':22, 'ruleg':23, 'rfoot':24}

aeroplane_labels = {'body': 1, 'stern': 2, 'lwing': 3, 'rwing':4, 'tail':5}
for ii in range(1, 10):
    aeroplane_labels['engine_{}'.format(ii)] = 5+ii
for ii in range(1, 10):
    aeroplane_labels['wheel_{}'.format(ii)] = 14+ii



motorbike_labels = {'fwheel': 1, 'bwheel': 2, 'handlebar': 3, 'saddle': 4}
for ii in range(0,10):
    motorbike_labels['headlight_{}'.format(ii+1)] = 5+ii
    motorbike_labels['body']=15



bicycle_labels = {'fwheel': 1, 'bwheel': 2, 'saddle': 3, 'handlebar': 4, 'chainwheel': 5}
for ii in range(0,10):
    bicycle_labels['headlight_{}'.format(ii+1)] = 6+ii
    bicycle_labels['body']=16

sheep_labels = cow_labels



part_labels = {'bird': bird_labels, 'cat': cat_labels, 'cow': cow_labels, 'dog': dog_labels, 'sheep': sheep_labels, 'horse':horse_labels, 'bicycle':bicycle_labels, 'motorbike':motorbike_labels, 'person':person_labels,'aeroplane':aeroplane_labels}

In [8]:
outfile = '/home/tanvikamble/MeronymNet-PyTorch/src/processed_data/X_train.np'
with open(outfile, 'rb') as pickle_file:
    X_train = pickle.load(pickle_file)

outfile = '/home/tanvikamble/MeronymNet-PyTorch/src/processed_data/class_v.np'
with open(outfile, 'rb') as pickle_file:
    class_v = pickle.load(pickle_file)

outfile = '/home/tanvikamble/MeronymNet-PyTorch/src/processed_data/adj_train.np'
with open(outfile, 'rb') as pickle_file:
    adj_train = pickle.load(pickle_file)

outfile = '/home/tanvikamble/MeronymNet-PyTorch/src/processed_data/X_train_val.np'
with open(outfile, 'rb') as pickle_file:
    X_train_val = pickle.load(pickle_file)

outfile = '/home/tanvikamble/MeronymNet-PyTorch/src/processed_data/class_v_val.np'
with open(outfile, 'rb') as pickle_file:
    class_v_val = pickle.load(pickle_file)
    
outfile = '/home/tanvikamble/MeronymNet-PyTorch/src/processed_data/adj_train_val.np'
with open(outfile, 'rb') as pickle_file:
    adj_train_val = pickle.load(pickle_file)

In [9]:
# for i in range(X_train.shape[0]):
#     for k in range(1,5):
#         X_train[i][0][k]=X_train[i][0][k]/canvas_size

In [10]:
# for i in range(X_train_val.shape[0]):
#     for k in range(1,5):
#         X_train_val[i][0][k]=X_train_val[i][0][k]/canvas_size

In [13]:
X_train[X_train<=0] = 0
X_train[X_train>=1] = 1
X_train_val[X_train_val<=0] = 0
X_train_val[X_train_val>=1] = 1

In [10]:
parts_present_per_class={}
class_dict = ['cow', 'sheep', 'bird', 'person', 'cat', 'dog', 'horse', 'aeroplane',
              'motorbike', 'bicycle']
for obj in class_dict:
    parts_present_per_class[obj] = np.zeros(24)

In [11]:
for i in range(X_train.shape[0]):
    obj = class_dict[int(np.argmax(class_v[i]))]
    for j in range(24):
        if(int(X_train[i][j][0]) == 1):
            parts_present_per_class[obj][j]=1 

In [12]:
adj_train_final = adj_train.copy()
for i in range(adj_train.shape[0]):
    parts_present=[]
    obj = class_dict[int(np.argmax(class_v[i]))]
    for j in range(24):
        if parts_present_per_class[obj][j] == 1:
            parts_present.append(j)
    for part_1 in parts_present:
        for part_2 in parts_present:
            adj_train_final[i][part_1][part_2]=1

In [13]:
parts_present_per_class_val={}
for obj in class_dict:
    parts_present_per_class_val[obj] = np.zeros(24)

for i in range(X_train_val.shape[0]):
    obj = class_dict[int(np.argmax(class_v_val[i]))]
    for j in range(24):
        if(int(X_train_val[i][j][0]) == 1):
            parts_present_per_class_val[obj][j]=1 

for obj in class_dict:
    print(parts_present_per_class_val[obj])

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0.]
[1. 1. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]


In [17]:
adj_train_val_final = adj_train_val.copy()
for i in range(adj_train_val.shape[0]):
    parts_present=[]
    obj = class_dict[int(np.argmax(class_v_val[i]))]
    for j in range(24):
        if parts_present_per_class_val[obj][j] == 1:
            parts_present.append(j)
    for part_1 in parts_present:
        for part_2 in parts_present:
            adj_train_val_final[i][part_1][part_2]=1

In [18]:
random.seed(100)
train_idx = np.random.randint(1,len(X_train),len(X_train))
test_idx = np.random.randint(1,len(X_train_val),len(X_train_val))

In [19]:
batch_size = 128
seed = 345
torch.manual_seed(seed)
train_list =[]
for idx, batch in enumerate(zip(copy.deepcopy(X_train[train_idx]),
                                copy.deepcopy(class_v[train_idx]),
                                copy.deepcopy(adj_train[train_idx]))):
    edge_index, _ = dense_to_sparse(torch.from_numpy(batch[2]).cuda().float())
    train_list.append(Data(x = torch.from_numpy(batch[0]).cuda().float(),
                           y = torch.from_numpy(batch[1]).cuda().float(),
                           edge_index = edge_index
                                )
                     )

batch_train_loader = DataLoader(train_list, batch_size=batch_size)
val_list = []
for idx, batch in enumerate(zip(copy.deepcopy(X_train_val[test_idx]),
                                copy.deepcopy(class_v_val[test_idx]), 
                                copy.deepcopy(adj_train_val[test_idx]))):
    edge_index, _ = dense_to_sparse(torch.from_numpy(batch[2]).cuda().float())
    val_list.append(Data(x = torch.from_numpy(batch[0]).cuda().float(),
                         y = torch.from_numpy(batch[1]).cuda().float(),
                         edge_index = edge_index
                                )
                     )
batch_val_loader = DataLoader(val_list, batch_size=batch_size)
del train_list
del val_list

In [20]:
batch_size = 128
seed = 345
torch.manual_seed(seed)
train_list =[]
for idx, batch in enumerate(zip(copy.deepcopy(X_train[train_idx]),
                                copy.deepcopy(class_v[train_idx]),
                                copy.deepcopy(adj_train_final[train_idx]))):
    edge_index, _ = dense_to_sparse(torch.from_numpy(batch[2]).cuda().float())
    train_list.append(Data(x = torch.from_numpy(batch[0]).cuda().float(),
                           y = torch.from_numpy(batch[1]).cuda().float(),
                           edge_index = edge_index
                                )
                     )

batch_train_loader_fc = DataLoader(train_list, batch_size=batch_size)
val_list = []
for idx, batch in enumerate(zip(copy.deepcopy(X_train_val[test_idx]),
                                copy.deepcopy(class_v_val[test_idx]), 
                                copy.deepcopy(adj_train_val_final[test_idx]))):
    edge_index, _ = dense_to_sparse(torch.from_numpy(batch[2]).cuda().float())
    val_list.append(Data(x = torch.from_numpy(batch[0]).cuda().float(),
                         y = torch.from_numpy(batch[1]).cuda().float(),
                         edge_index = edge_index
                                )
                     )
batch_val_loader_fc = DataLoader(val_list, batch_size=batch_size)
del train_list
del val_list

In [21]:
idx = 0
for data in batch_train_loader:
    idx+=1
    print(data.x.shape)
    if idx==3:
        break

torch.Size([3072, 5])
torch.Size([3072, 5])
torch.Size([3072, 5])


In [22]:
idx = 0
for data in batch_train_loader_fc:
    idx+=1
    print(data.x.shape)
    if idx==3:
        break

torch.Size([3072, 5])
torch.Size([3072, 5])
torch.Size([3072, 5])


In [23]:
# change num_nodes and num_classes whenever running this code for the modification experiments

latent_dims = 64
batch_size = 128
num_nodes = 24
bbx_size = 4
num_classes = 10
label_shape = 1
nb_epochs = 250
klw = loss.frange_cycle_linear(nb_epochs)
learning_rate = 0.000065
hidden1 = 32
hidden2 = 16
hidden3 = 128
run_prefix = "Comparison-6"

In [24]:
import gc
gc.collect()

0

In [25]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [26]:
# # x_minp, y_minp,x_maxp , y_maxp= coord
# # cell for checking symmetry of the boxes 
# #testing loop

# final_box_dict = {}
# model_path_1 = ('/home/tanvikamble/MeronymNet-PyTorch/src/model/'+'only_torso_experiment_margin_in_loss_sum'+'/GCN-lr-'
#                         +str(learning_rate)
#                         +'-batch-'+str(batch_size)
#                         +'-h1-'+str(hidden1)
#                         +'-h2-'+str(hidden2)
#                         +'-h3-'+str(hidden3)+'-test')

# class_dict = ['cow', 'sheep', 'bird', 'person', 'cat', 'dog', 'horse', 'aeroplane']
# count_dict = {'cow':0, 'sheep':0, 'bird':0, 'person':0, 'cat':0, 'dog':0, 'horse':0}
# image_shape = [num_nodes, bbx_size]
# global_step = 250000
# # changed dynmaic margin for this to false. be careful. For the model 1. Change later
# vae_1 = GCNAutoEncoder_Combined_Parts(latent_dims,num_nodes,bbx_size,num_classes,label_shape,hidden1, hidden2, hidden3, dynamic_margin=True)
# vae_1.load_state_dict(torch.load(model_path_1+ '/model_weights.pth'))
# decoder_1 = vae_1.decoder

# for i, val_data in enumerate(batch_val_loader, 0):
    
#     val_data.cuda()
#     node_data_true = val_data.x
    
#     label_true = node_data_true[:,:1]
#     class_true = val_data.y
#     val_batch_size = int(class_true.shape[0]/7)
#     adj_true = val_data.edge_index
# #     output = inference(decoder, label_true , class_true, latent_dims, val_batch_size)
#     # First Model
#     output_1 = vae_1(adj_true, node_data_true, label_true , class_true)
#     node_data_pred_test_1 = output_1[0]
#     print(node_data_pred_test_1.shape)
#     node_data_pred_test_1 = node_data_pred_test_1.to('cpu')
#     node_data_pred_test_1 = node_data_pred_test_1.detach().numpy()
#     node_data_pred_test_1 = node_data_pred_test_1.reshape(node_data_pred_test_1.shape[0], node_data_pred_test_1.shape[2])
#     for j in range(node_data_pred_test_1.shape[0]):
#         class_i = class_dict[int(np.argmax(class_true[7*j:7*(j+1)].detach().to('cpu').numpy()).tolist())]
#         if class_i in final_box_dict:
#             final_box_dict[class_i].append(node_data_pred_test_1[j])
#         else:
#             print(class_i)
#             final_box_dict[class_i] = [node_data_pred_test_1[j]]

In [27]:
# from prettytable import PrettyTable
# # x_minp, y_minp,x_maxp , y_maxp= coord
# k=0

# myTable_x = PrettyTable(["Class", "0", "1", "2", "3", "4"])
# myTable_y = PrettyTable(["Class", "0", "1", "2", "3", "4"])
# myTable_x.title = 'Number of Images along x direction:VALIDATION'
# myTable_y.title = 'Number of Images along y direction:VALIDATION'
# for class_i in final_box_dict:
#     print(class_i)
#     print(len(final_box_dict[class_i]))
#     diff_x = [0 for i in range(5)]
#     diff_y = [0 for i in range(5)]
    
#     for box in final_box_dict[class_i]:
#         box=box*660
#         box=box.astype(int)
#         box= box-330
#         box=abs(box)
#         diff_x[abs(box[0]-box[2])]+=1
#         diff_y[abs(box[1]-box[3])]+=1
#     myTable_x.add_row([str(class_i), str(diff_x[0]),str(diff_x[1]),str(diff_x[2]),str(diff_x[3]),str(diff_x[4])])
#     myTable_y.add_row([str(class_i), str(diff_y[0]),str(diff_y[1]),str(diff_y[2]),str(diff_y[3]),str(diff_y[4])])
   
 

In [28]:
# print(myTable_x)

In [29]:
# print(myTable_y)

In [30]:
# summary_path = ('/home/tanvikamble/MeronymNet-PyTorch/src/runs/'+run_prefix+'/GCN-lr-'
#                         +str(learning_rate)
#                         +'-batch-'+str(batch_size)
#                         +'-h1-'+str(hidden1)
#                         +'-h2-'+str(hidden2)
#                         +'-h3-'+str(hidden3)+'-test')
# write_tensorboard = True
# if write_tensorboard:
#     writer = SummaryWriter(summary_path)


In [31]:
iou_loss_dict_1={}
mse_loss_dict_1={}

iou_loss_dict_final_1={}
mse_loss_dict_final_1={}

iou_loss_dict_2={}
mse_loss_dict_2={}

iou_loss_dict_final_2={}
mse_loss_dict_final_2={}

# iou_loss_dict_3={}
# mse_loss_dict_3={}

# iou_loss_dict_final_3={}
# mse_loss_dict_final_3={}

# iou_loss_dict_4={}
# mse_loss_dict_4={}

# iou_loss_dict_final_4={}
# mse_loss_dict_final_4={}

labels_dict={}
coords_dict={}


In [32]:
# iou_loss_dict_3.clear()
# mse_loss_dict_3.clear()
# iou_loss_dict_4.clear()
# mse_loss_dict_4.clear()

#testing loop
model_path_1 = ('/home/tanvikamble/MeronymNet-PyTorch/src/model/'+'Reconstruction_Fully_Connected_No_Margin_Verson-2'+'/GCN-lr-'
                        +str(learning_rate)
                        +'-batch-'+str(batch_size)
                        +'-h1-'+str(hidden1)
                        +'-h2-'+str(hidden2)
                        +'-h3-'+str(hidden3)+'-test')

model_path_2 = ('/home/tanvikamble/MeronymNet-PyTorch/src/model/'+'Reconstruction_Without_Margin'+'/GCN-lr-'
                        +str(learning_rate)
                        +'-batch-'+str(batch_size)
                        +'-h1-'+str(hidden1)
                        +'-h2-'+str(hidden2)
                        +'-h3-'+str(hidden3)+'-test')


# model_path_3 = ('/home/tanvikamble/MeronymNet-PyTorch/src/model/'+'Rec_Margin_In_Loss_Mean_Combined_Parts-1'+'/GCN-lr-'
#                         +str(learning_rate)
#                         +'-batch-'+str(batch_size)
#                         +'-h1-'+str(hidden1)
#                         +'-h2-'+str(hidden2)
#                         +'-h3-'+str(hidden3)+'-test')



# model_path_4 = ('/home/tanvikamble/MeronymNet-PyTorch/src/model/'+'Rec_Margin_Not_In_Loss_Combined_Parts-1'+'/GCN-lr-'
#                         +str(learning_rate)
#                         +'-batch-'+str(batch_size)
#                         +'-h1-'+str(hidden1)
#                         +'-h2-'+str(hidden2)
#                         +'-h3-'+str(hidden3)+'-test')

class_dict = ['cow', 'sheep', 'bird', 'person', 'cat', 'dog', 'horse', 'aeroplane', 'motorbike' , 'bicycle']
image_shape = [num_nodes, bbx_size]
global_step = 250000
# changed dynmaic margin for this to false. be careful. For the model 1. Change later
vae_1 = GCNAutoEncoder(latent_dims,num_nodes,bbx_size,num_classes,label_shape,hidden1, hidden2, hidden3, dynamic_margin=False)
vae_1.load_state_dict(torch.load(model_path_1+ '/model_weights.pth'))

vae_2 = GCNAutoEncoder(latent_dims,num_nodes,bbx_size,num_classes,label_shape,hidden1, hidden2, hidden3, dynamic_margin=False)
vae_2.load_state_dict(torch.load(model_path_2+ '/model_weights.pth'))

# vae_3 = GCNAutoEncoder_Combined_Parts(latent_dims,num_nodes,bbx_size,num_classes,label_shape,hidden1, hidden2, hidden3, dynamic_margin=True)
# vae_3.load_state_dict(torch.load(model_path_3+ '/model_weights.pth'))

# vae_4 = GCNAutoEncoder_Combined_Parts(latent_dims,num_nodes,bbx_size,num_classes,label_shape,hidden1, hidden2, hidden3, dynamic_margin=True)
# vae_4.load_state_dict(torch.load(model_path_4+ '/model_weights.pth'))

decoder_1 = vae_1.decoder
decoder_2 = vae_2.decoder
# decoder_3= vae_3.decoder
# decoder_4 = vae_4.decoder



In [50]:
iou_loss_dict_1.clear()
mse_loss_dict_1.clear()
for i, val_data in enumerate(batch_train_loader_fc, 0):
    
    val_data.cuda()
    node_data_true = val_data.x
    
    label_true = node_data_true[:,:1]
    class_true = val_data.y
    val_batch_size = int(class_true.shape[0]/10)
    adj_true = val_data.edge_index
#     output = inference(decoder, label_true , class_true, latent_dims, val_batch_size)
    # First Model
    output_1 = vae_1(adj_true, node_data_true, label_true , class_true)
    node_data_pred_test_1 = output_1[0]
    iou_loss_1,mse_loss_1=loss.calc_losses(node_data_pred_test_1, node_data_true[:,1:])
    iou_loss_1 = iou_loss_1.to('cpu')
    iou_loss_1 = iou_loss_1.detach().numpy() 
    mse_loss_1 = mse_loss_1.to('cpu')
    mse_loss_1 = mse_loss_1.detach().numpy()
#     print(mse_loss_1.shape)
#     # Second Model
#     output_2 = vae_2(adj_true, node_data_true, label_true , class_true)
#     node_data_pred_test_2 = output_2[0]
#     iou_loss_2,mse_loss_2=loss.calc_losses(node_data_pred_test_2, node_data_true[:,1:])
    
#     iou_loss_2 = iou_loss_2.to('cpu')
#     iou_loss_2 = iou_loss_2.detach().numpy()
#     mse_loss_2 = mse_loss_2.to('cpu')
#     mse_loss_2 = mse_loss_2.detach().numpy() 
#     # Third Model
#     output_3 = vae_3(adj_true, node_data_true, label_true , class_true)
#     node_data_pred_test_3 = output_3[0]
#     iou_loss_3,mse_loss_3=loss.calc_losses(node_data_pred_test_3, node_data_true[:,1:])
#     iou_loss_3 = iou_loss_3.to('cpu')
#     iou_loss_3 = iou_loss_3.detach().numpy()
#     mse_loss_3 = mse_loss_3.to('cpu')
#     mse_loss_3 = mse_loss_3.detach().numpy() 
#     # Fourth Model
#     output_4 = vae_4(adj_true, node_data_true, label_true , class_true)
#     node_data_pred_test_4 = output_4[0]
#     iou_loss_4,mse_loss_4=loss.calc_losses(node_data_pred_test_4, node_data_true[:,1:])
#     iou_loss_4 = iou_loss_4.to('cpu')
#     iou_loss_4 = iou_loss_4.detach().numpy()
#     mse_loss_4 = mse_loss_4.to('cpu')
#     mse_loss_4 = mse_loss_4.detach().numpy() 
    
 
    
    

    label_true_reshaped = label_true.reshape((iou_loss_1.shape[0],iou_loss_1.shape[1]))
    label_true_reshaped = label_true_reshaped.to('cpu')
    label_true_reshaped = label_true_reshaped.detach().numpy()
    
    
    
    node_data_true_reshaped = node_data_true.reshape((iou_loss_1.shape[0],iou_loss_1.shape[1] , node_data_true.shape[1]))
    node_data_true_reshaped = node_data_true_reshaped.to('cpu')
    node_data_true_reshaped = node_data_true_reshaped.detach().numpy()
    
    for j in range(iou_loss_1.shape[0]):
        class_i = class_dict[int(np.argmax(class_true[10*j:10*(j+1)].detach().to('cpu').numpy()).tolist())]
        if class_i in iou_loss_dict_1:
            labels_dict[class_i].append(label_true_reshaped[j])
            coords_dict[class_i].append(node_data_true_reshaped[j])
            iou_loss_dict_1[class_i].append(iou_loss_1[j])
            mse_loss_dict_1[class_i].append(mse_loss_1[j])
#             # second model
#             iou_loss_dict_2[class_i].append(iou_loss_2[j])
#             mse_loss_dict_2[class_i].append(mse_loss_2[j])
#             # third model
#             iou_loss_dict_3[class_i].append(iou_loss_3[j])
#             mse_loss_dict_3[class_i].append(mse_loss_3[j])
#             # fourth model
#             iou_loss_dict_4[class_i].append(iou_loss_4[j])
#             mse_loss_dict_4[class_i].append(mse_loss_4[j])
        else:
            print(class_i)
            labels_dict[class_i]=  [label_true_reshaped[j]]
            coords_dict[class_i] = [node_data_true_reshaped[j]]
            iou_loss_dict_1[class_i]=[iou_loss_1[j]]
            mse_loss_dict_1[class_i]=[mse_loss_1[j]]
#             print(class_i + " done in Dict-1" )
#             # second model
#             iou_loss_dict_2[class_i]=[iou_loss_2[j]]
#             mse_loss_dict_2[class_i]=[mse_loss_2[j]]
#             print(class_i + " done in Dict-2" )
#             # third model
#             iou_loss_dict_3[class_i]=[iou_loss_3[j]]
#             mse_loss_dict_3[class_i]=[mse_loss_3[j]]
#             print(class_i + " done in Dict-3" )
#             # fourth model
#             iou_loss_dict_4[class_i]=[iou_loss_4[j]]
#             mse_loss_dict_4[class_i]=[mse_loss_4[j]]
#             print(class_i + " done in Dict-4" )
#             print("xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx")

    


aeroplane
sheep
bird
dog
person
cat
horse
cow
bicycle
motorbike


In [51]:
iou_loss_dict_2.clear()
mse_loss_dict_2.clear()

for i, val_data in enumerate(batch_train_loader, 0):
    
    val_data.cuda()
    node_data_true = val_data.x
    
    label_true = node_data_true[:,:1]
    class_true = val_data.y
    val_batch_size = int(class_true.shape[0]/7)
    adj_true = val_data.edge_index
#     output = inference(decoder, label_true , class_true, latent_dims, val_batch_size)
    # First Model
#     output_1 = vae_1(adj_true, node_data_true, label_true , class_true)
#     node_data_pred_test_1 = output_1[0]
#     iou_loss_1,mse_loss_1=loss.calc_losses(node_data_pred_test_1, node_data_true[:,1:])
#     iou_loss_1 = iou_loss_1.to('cpu')
#     iou_loss_1 = iou_loss_1.detach().numpy() 
#     mse_loss_1 = mse_loss_1.to('cpu')
#     mse_loss_1 = mse_loss_1.detach().numpy()
#     print(mse_loss_1.shape)
    # Second Model
    output_2 = vae_2(adj_true, node_data_true, label_true , class_true)
    node_data_pred_test_2 = output_2[0]
    iou_loss_2,mse_loss_2=loss.calc_losses(node_data_pred_test_2, node_data_true[:,1:])
    
    iou_loss_2 = iou_loss_2.to('cpu')
    iou_loss_2 = iou_loss_2.detach().numpy()
    mse_loss_2 = mse_loss_2.to('cpu')
    mse_loss_2 = mse_loss_2.detach().numpy() 
#     # Third Model
#     output_3 = vae_3(adj_true, node_data_true, label_true , class_true)
#     node_data_pred_test_3 = output_3[0]
#     iou_loss_3,mse_loss_3=loss.calc_losses(node_data_pred_test_3, node_data_true[:,1:])
#     iou_loss_3 = iou_loss_3.to('cpu')
#     iou_loss_3 = iou_loss_3.detach().numpy()
#     mse_loss_3 = mse_loss_3.to('cpu')
#     mse_loss_3 = mse_loss_3.detach().numpy() 
#     # Fourth Model
#     output_4 = vae_4(adj_true, node_data_true, label_true , class_true)
#     node_data_pred_test_4 = output_4[0]
#     iou_loss_4,mse_loss_4=loss.calc_losses(node_data_pred_test_4, node_data_true[:,1:])
#     iou_loss_4 = iou_loss_4.to('cpu')
#     iou_loss_4 = iou_loss_4.detach().numpy()
#     mse_loss_4 = mse_loss_4.to('cpu')
#     mse_loss_4 = mse_loss_4.detach().numpy() 
    for j in range(iou_loss_2.shape[0]):
        class_i = class_dict[int(np.argmax(class_true[10*j:10*(j+1)].detach().to('cpu').numpy()).tolist())]
        if class_i in iou_loss_dict_2:
#             iou_loss_dict_1[class_i].append(iou_loss_1[j])
#             mse_loss_dict_1[class_i].append(mse_loss_1[j])
            # second model
            iou_loss_dict_2[class_i].append(iou_loss_2[j])
            mse_loss_dict_2[class_i].append(mse_loss_2[j])
#             # third model
#             iou_loss_dict_3[class_i].append(iou_loss_3[j])
#             mse_loss_dict_3[class_i].append(mse_loss_3[j])
#             # fourth model
#             iou_loss_dict_4[class_i].append(iou_loss_4[j])
#             mse_loss_dict_4[class_i].append(mse_loss_4[j])
        else:
            print(class_i)
#             labels_dict[class_i]=  [label_true_reshaped[j]]
#             coords_dict[class_i] = [node_data_true_reshaped[j]]
#             iou_loss_dict_1[class_i]=[iou_loss_1[j]]
#             mse_loss_dict_1[class_i]=[mse_loss_1[j]]
#             print(class_i + " done in Dict-1" )
            # second model
            iou_loss_dict_2[class_i]=[iou_loss_2[j]]
            mse_loss_dict_2[class_i]=[mse_loss_2[j]]
#             print(class_i + " done in Dict-2" )
#             # third model
#             iou_loss_dict_3[class_i]=[iou_loss_3[j]]
#             mse_loss_dict_3[class_i]=[mse_loss_3[j]]
#             print(class_i + " done in Dict-3" )
#             # fourth model
#             iou_loss_dict_4[class_i]=[iou_loss_4[j]]
#             mse_loss_dict_4[class_i]=[mse_loss_4[j]]
#             print(class_i + " done in Dict-4" )
#             print("xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx")

    


aeroplane
sheep
bird
dog
person
cat
horse
cow
bicycle
motorbike


In [52]:
for class_i in labels_dict:
    print(len(labels_dict[class_i]))
    print(len(coords_dict[class_i]))
    print(len(iou_loss_dict_1[class_i]))
    print(len(iou_loss_dict_2[class_i]))
#     print(len(iou_loss_dict_3[class_i]))
#     print(len(iou_loss_dict_4[class_i]))
    print(len(mse_loss_dict_1[class_i]))
    print(len(mse_loss_dict_2[class_i]))
#     print(len(mse_loss_dict_3[class_i]))
#     print(len(mse_loss_dict_4[class_i]))
    print("xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx")

9246
9246
9246
9246
9246
9246
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
9099
9099
9099
9099
9099
9099
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
5182
5182
5182
5182
5182
5182
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
8624
8624
8624
8624
8624
8624
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
10218
10218
10218
10218
10218
10218
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
8124
8124
8124
8124
8124
8124
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
8825
8825
8825
8825
8825
8825
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
6546
6546
6546
6546
6546
6546
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
10544
10544
10544
10544
10544
10544
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
3962
3962
3962
3962
3962
3962
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx


In [53]:
parts=[i for i in range(24)]

In [54]:
counts_dict_final={}
for class_i in labels_dict:
    temp_list=np.zeros(24)
    for part_num in parts:
        temp_count=0
        for ele in labels_dict[class_i]:
            temp_count+=int(ele[part_num])
        temp_list[part_num]=temp_count
    counts_dict_final[class_i]=temp_list
    

In [55]:
import pandas as pd
stats_dict_final={}
for class_i in coords_dict:
    final_list=[[] for i in range(24)]
    for part_num in parts:
        temp_list=[[] for i in range(24)]
        for ele in coords_dict[class_i]:
            temp_tuple=ele[part_num]
            if int(temp_tuple[0]) != 0:
                temp_list[part_num].append(abs(temp_tuple[3]-temp_tuple[1]) * abs(temp_tuple[4]-temp_tuple[2]))
        s=pd.Series(temp_list[part_num], dtype='float64')
        final_list[part_num].append(s.mean())
        final_list[part_num].append(s.quantile(q=0.25))
        final_list[part_num].append(s.quantile(q=0.50))
        final_list[part_num].append(s.quantile(q=0.75))
    stats_dict_final[class_i] = final_list

In [56]:
side_ratios_dict_final={}
for class_i in coords_dict:
    final_list=[0 for i in range(24)]
    for part_num in parts:
        temp_list=[[] for i in range(24)]
        for ele in coords_dict[class_i]:
            temp_tuple=ele[part_num]
            if int(temp_tuple[0]) != 0:
                temp_ratio=(abs(temp_tuple[3]-temp_tuple[1])*10000)/(abs(temp_tuple[4]-temp_tuple[2])*10000)
                if temp_ratio < 1:
                    temp_ratio = 1/temp_ratio 
                temp_list[part_num].append(temp_ratio)
        s=pd.Series(temp_list[part_num], dtype='float64')
        final_list[part_num]=s.mean()
    side_ratios_dict_final[class_i] = final_list

/tmp/ipykernel_7672/3554822654.py:9: RuntimeWarning:

invalid value encountered in double_scalars

/tmp/ipykernel_7672/3554822654.py:9: RuntimeWarning:

divide by zero encountered in double_scalars

/tmp/ipykernel_7672/3554822654.py:11: RuntimeWarning:

divide by zero encountered in double_scalars



In [57]:
for class_i in stats_dict_final:
#     print(len(stats_dict_final[class_i]))
#     print(len(counts_dict_final[class_i]))
    print(len(stats_dict_final[class_i]))
    print(len(counts_dict_final[class_i]))
    print(len(side_ratios_dict_final[class_i]))
    print("xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx")
   
    

24
24
24
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
24
24
24
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
24
24
24
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
24
24
24
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
24
24
24
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
24
24
24
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
24
24
24
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
24
24
24
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
24
24
24
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
24
24
24
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx


In [58]:
def dict_summation(loss_dict,counts_dict_final):
    loss_dict_final={}
    parts=[i for i in range(24)]
    for class_i in loss_dict:
        temp_list=np.zeros(24)
        for part_num in parts:
            temp_sum=0
#             temp_non_zero_elements=0
            temp_non_zero_elements=counts_dict_final[class_i][part_num]
            for ele in loss_dict[class_i]:
                temp_sum+=ele[part_num]
            if temp_non_zero_elements != 0:
                temp_list[part_num] = temp_sum/temp_non_zero_elements
            else:
                temp_list[part_num] = temp_sum
            loss_dict_final[class_i] = temp_list
    return loss_dict_final


    

In [59]:
iou_loss_dict_final_1=dict_summation(iou_loss_dict_1,counts_dict_final)
iou_loss_dict_final_2=dict_summation(iou_loss_dict_2,counts_dict_final)
# iou_loss_dict_final_3=dict_summation(iou_loss_dict_3,counts_dict_final)
# iou_loss_dict_final_4=dict_summation(iou_loss_dict_4,counts_dict_final)

In [60]:
mse_loss_dict_final_1=dict_summation(mse_loss_dict_1,counts_dict_final)
mse_loss_dict_final_2=dict_summation(mse_loss_dict_2,counts_dict_final)
# mse_loss_dict_final_3=dict_summation(mse_loss_dict_3,counts_dict_final)
# mse_loss_dict_final_4=dict_summation(mse_loss_dict_4,counts_dict_final)

In [61]:
for class_i in iou_loss_dict_final_1:
    for part in range(24):
        iou_loss_dict_final_1[class_i][part] = round(iou_loss_dict_final_1[class_i][part],3)
        iou_loss_dict_final_2[class_i][part] = round(iou_loss_dict_final_2[class_i][part],3)
#         iou_loss_dict_final_3[class_i][part] = round(iou_loss_dict_final_3[class_i][part],3)
#         iou_loss_dict_final_4[class_i][part] = round(iou_loss_dict_final_4[class_i][part],3)
        
        mse_loss_dict_final_1[class_i][part] = mse_loss_dict_final_1[class_i][part]*(660*660)
        mse_loss_dict_final_1[class_i][part] = round(mse_loss_dict_final_1[class_i][part],1)
        
        
        
        
        mse_loss_dict_final_2[class_i][part] = mse_loss_dict_final_2[class_i][part]*(660*660)
        mse_loss_dict_final_2[class_i][part] = round(mse_loss_dict_final_2[class_i][part],1)
        
        
#         mse_loss_dict_final_3[class_i][part] = mse_loss_dict_final_3[class_i][part]*(660*660)
#         mse_loss_dict_final_3[class_i][part] = round(mse_loss_dict_final_3[class_i][part],1)
        
        
#         mse_loss_dict_final_4[class_i][part] = mse_loss_dict_final_4[class_i][part]*(660*660)
#         mse_loss_dict_final_4[class_i][part] = round(mse_loss_dict_final_4[class_i][part],1)
        
        side_ratios_dict_final[class_i][part] = round(side_ratios_dict_final[class_i][part],3)
        for i in range(4):
            stats_dict_final[class_i][part][i] = round(stats_dict_final[class_i][part][i],3)
        
        
    

In [62]:
for class_i in mse_loss_dict_final_1:
    print(iou_loss_dict_final_1[class_i])

[0.577 0.384 0.36  0.207 0.206 0.287 0.198 0.165 0.118 0.    0.    0.
 0.    0.    0.095 0.08  0.055 0.027 0.011 0.023 0.013 0.009 0.    0.   ]
[0.7   0.078 0.071 0.325 0.315 0.477 0.258 0.307 0.725 0.536 0.281 0.239
 0.284 0.256 0.289 0.185 0.285 0.265 0.271 0.    0.    0.    0.    0.   ]
[0.564 0.078 0.088 0.291 0.656 0.431 0.324 0.285 0.273 0.22  0.29  0.22
 0.407 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.   ]
[0.721 0.126 0.133 0.408 0.409 0.222 0.658 0.446 0.403 0.261 0.403 0.287
 0.361 0.149 0.334 0.135 0.249 0.587 0.    0.    0.    0.    0.    0.   ]
[0.651 0.14  0.136 0.208 0.229 0.183 0.159 0.314 0.192 0.509 0.669 0.48
 0.34  0.437 0.137 0.315 0.399 0.165 0.361 0.458 0.231 0.39  0.5   0.258]
[0.757 0.248 0.25  0.44  0.439 0.221 0.685 0.461 0.448 0.267 0.452 0.279
 0.36  0.176 0.351 0.151 0.327 0.    0.    0.    0.    0.    0.    0.   ]
[0.679 0.087 0.095 0.264 0.264 0.555 0.079 0.089 0.74  0.592 0.369 0.25
 0.389 0.273 0.354 0.244 0.335 0.272 0.327 0.093 0.

In [63]:
print("hi")

hi


In [64]:
def class_iou(loss_dict_final,counts_dict_final,current_class):
    total_count=0
    total_val=0
    for part in range(24):
        total_val+=loss_dict_final[current_class][part] * counts_dict_final[current_class][part]
        total_count+=counts_dict_final[current_class][part]
    total_val=total_val/total_count
    return total_val
        

In [65]:
print(iou_loss_dict_final_1)

{'aeroplane': array([0.577, 0.384, 0.36 , 0.207, 0.206, 0.287, 0.198, 0.165, 0.118,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.095, 0.08 , 0.055, 0.027,
       0.011, 0.023, 0.013, 0.009, 0.   , 0.   ]), 'sheep': array([0.7  , 0.078, 0.071, 0.325, 0.315, 0.477, 0.258, 0.307, 0.725,
       0.536, 0.281, 0.239, 0.284, 0.256, 0.289, 0.185, 0.285, 0.265,
       0.271, 0.   , 0.   , 0.   , 0.   , 0.   ]), 'bird': array([0.564, 0.078, 0.088, 0.291, 0.656, 0.431, 0.324, 0.285, 0.273,
       0.22 , 0.29 , 0.22 , 0.407, 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.   ]), 'dog': array([0.721, 0.126, 0.133, 0.408, 0.409, 0.222, 0.658, 0.446, 0.403,
       0.261, 0.403, 0.287, 0.361, 0.149, 0.334, 0.135, 0.249, 0.587,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.   ]), 'person': array([0.651, 0.14 , 0.136, 0.208, 0.229, 0.183, 0.159, 0.314, 0.192,
       0.509, 0.669, 0.48 , 0.34 , 0.437, 0.137, 0.315, 0.399, 0.165,
       0.361, 0.458, 0.231, 0.39 , 0.5  , 0.258]

In [66]:
# # print(myTable)
# # bird
# # person
# # cat
# # horse
# # cow
# # dog
# # sheep
# import plotly.graph_objects as go
# import pandas as pd
# # df = pd.DataFrame(data)
# class_list=["person","bird", "cat" , "horse", "cow", "dog", "sheep"]
# mean_list_final = []
# median_list_final = []
# counts_list=[]
# ratios_list=[]
# temp_list_1=[]
# for current_class in class_list:
#     for i in stats_dict_final[current_class]:
#         mean_list_final.append(i[0])
#         median_list_final.append(i[2])
#     counts_list.append(counts_dict_final[current_class])
#     ratios_list.append(list(side_ratios_dict_final[current_class]))
#     temp_list_1.append(list(iou_loss_dict_final_1[current_class]))
    
# data={
#     "Classes": class_list,
#     "Count" : counts_list,
#     "Mean_Area": mean_list_final,
#     "Median_Area": median_list_final,
#     "Avg_side_ratio": ratios_list,
#     "Margin in Loss- SUM": temp_list_1
# }
# df = pd.DataFrame(data)


# fig = go.Figure(data=[go.Table(
#     header=dict(values=list(df.columns),
#                 fill_color='paleturquoise',
#                 align='left'),
#     cells=dict(values=df.transpose().values.tolist(),
#                fill_color='lavender',
#                align='left'))
# ])

# fig.update_layout(title= "IOU Values for Torso: VALIDATION",height=4000)
# # fig.title(current_class + ": iou Values")
# fig.show()

In [67]:
# print(myTable)
# bird
#person
# cat
# horse
# cow
# dog
# sheep
import plotly.graph_objects as go
import pandas as pd
# df = pd.DataFrame(data)
current_class="person"
mean_list_final = []
median_list_final = []
for i in stats_dict_final[current_class]:
    mean_list_final.append(i[0])
    median_list_final.append(i[2])

part_list=list(part_labels[current_class].keys())
while len(part_list) != 24:
    part_list.append("NA")
part_list.append("Mean iou of Class")
counts_list=list(counts_dict_final[current_class])
counts_list.append(0)
mean_list_final.append(0)
median_list_final.append(0)

ratios_list=list(side_ratios_dict_final[current_class])
ratios_list.append(0)


temp_list_1=list(iou_loss_dict_final_1[current_class])
temp_list_1.append(round(class_iou(iou_loss_dict_final_1,counts_dict_final,current_class),3))
temp_list_2=list(iou_loss_dict_final_2[current_class])
temp_list_2.append(round(class_iou(iou_loss_dict_final_2,counts_dict_final,current_class),3))
# temp_list_3=list(iou_loss_dict_final_3[current_class])
# temp_list_3.append(round(class_iou(iou_loss_dict_final_3,counts_dict_final,current_class),3))
# temp_list_4=list(iou_loss_dict_final_4[current_class])
# temp_list_4.append(round(class_iou(iou_loss_dict_final_4,counts_dict_final,current_class),3))

data={
    "Part": part_list,
    "Count" : counts_list,
    "Mean_Area": mean_list_final,
    "Median_Area": median_list_final,
    "Avg_side_ratio": ratios_list,
    "Fully Connected : No Margin": temp_list_1,
    "No Margin": temp_list_2,
#     "Margin in Loss(MEAN)": temp_list_3,
#     "Margin not in Loss": temp_list_4
}
df = pd.DataFrame(data)

df_new=df.sort_values(by=['Median_Area'], ascending=False)

fig = go.Figure(data=[go.Table(
    header=dict(values=list(df_new.columns),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=df_new.transpose().values.tolist(),
               fill_color='lavender',
               align='left'))
])

fig.update_layout(title= current_class + ": iou_Values",height=4000)
# fig.title(current_class + ": iou Values")
fig.show()